# Coverage of ISIC sectors (high level) by EX sectors and EI activities
based on bridge matrix and ecoinvent ISIC classfications

In [1]:
import pandas as pd
import pickle
from itertools import islice

pd.options.display.float_format = '{:.3g}'.format

# Load data

In [2]:
scenario = '2019-12-02_WW'  # must be the same as the HIOT Excel sheetname

In [3]:
DATA_PATH = "../data/"
SCENARIO_PATH = "../results/" + scenario + "/"
print(SCENARIO_PATH)

../results/2019-12-02_WW/


In [4]:
def load_pickle(filename):
    with open(filename, 'rb') as handle:
        data = pickle.load(handle)
        print(len(data))
        if isinstance(data, dict):
            print(list(islice(data.items(), 2)))
    return data

### File names

In [5]:
# ecoinvent
activity_overview = f"{DATA_PATH}ecoinvent/activity_overview_3.4_cut-off_LCIA_results.xlsx"

# EXIOBASE
hiot_results = f"{DATA_PATH}exiobase/hiot_results_PTM_v2.xlsx"
hiot_results_sheet = scenario

# Matching
matching_file = f"{DATA_PATH}matching/Matching file.xlsx"
product_matching_sheet = 'product matching'
process_matching_sheet = 'process matching electricity'
region_matching_sheet = "regional matching"

# ISIC classifications

### ISIC file
This file includes isic classifications level 1-4 plus additionally level 5 classifications from ecoinvent

In [6]:
ISIC = pd.read_excel(f"{DATA_PATH}matching/ISIC.xlsx", sheet_name='ISIC4')
print(ISIC.shape)
ISIC.head(2)

(783, 18)


,Code,Description,Level 1,Level 2,Level 3,Level 4,Level 5,Top category,Code 1,Code 2,Code 3,Code 4,Code 5,Description 1,Description 2,Description 3,Description 4,Description 5
0,A,"Agriculture, forestry and fishing",1,0,0,0,0,A,A,NaN,NaN,NaN,NaN,"Agriculture, forestry and fishing",NaN,NaN,NaN,NaN
1,01,"Crop and animal production, hunting and relate...",0,1,0,0,0,A,A,1,NaN,NaN,NaN,"Agriculture, forestry and fishing","Crop and animal production, hunting and relate...",NaN,NaN,NaN


In [7]:
def get_ISIC_levelX(description, level=1, debug=False):
    """Return a higher or lower level ISIC names and codes, if possible."""
    lookup_columns = ['Description 1', 'Description 2', 'Description 3', 'Description 4', 'Description 5']
    result = None
    
    for col in lookup_columns:
        # print(col)
        if description in ISIC[col].tolist():
            indices = ISIC[col].index[ISIC[col] == description].tolist()
            output_level = 'Description ' + str(level)
            result = set(ISIC.iloc[indices][output_level].tolist())
            if debug:
                print('Found in', col)
                print('Indices:', ISIC[col].index[ISIC[col] == description].tolist())
                print('Level', level, 'match:', result)
       
    return result

In [8]:
d5 = ' Liquid and gaseous fuels from biomass'
d3 = 'Growing of non-perennial crops'
d2 = 'Crop and animal production, hunting and related service activities'
d = 'Agriculture, forestry and fishing'
get_ISIC_levelX(d5, level=2, debug=True)

Found in Description 5
Indices: [162]
Level 2 match: {'Manufacture of coke and refined petroleum products'}


{'Manufacture of coke and refined petroleum products'}

# Results template (ISIC level 1)

In [9]:
DF = ISIC[['Code', 'Description']].loc[ISIC['Level 1'] == 1]
DF.reset_index(inplace=True, drop=True)
DF

,Code,Description
0,A,"Agriculture, forestry and fishing"
1,B,Mining and quarrying
2,C,Manufacturing
3,D,"Electricity, gas, steam and air conditioning s..."
4,E,"Water supply; sewerage, waste management and r..."
5,F,Construction
6,G,Wholesale and retail trade; repair of motor ve...
7,H,Transportation and storage
8,I,Accommodation and food service activities
9,J,Information and communication


# Ecoinvent

### ecoinvent activity overview

In [10]:
AO = pd.read_excel(f"{DATA_PATH}ecoinvent/activity_overview_3.4_cut-off_LCIA_results_with_ISIC_details.xlsx")
#ao.set_index(0, inplace=True)
print(AO.shape)
AO = AO[AO['specialActivityType'] == 'ordinary transforming activity']
print(AO.shape)
AO.head(2)

(14889, 58)
(10805, 58)


,Unnamed: 0,Unnamed: 0_x,key,#,id,filename,activityName,geography,startDate,endDate,...,Description 1,Description 2,Description 3,Description 4,Description 5,ISIC 1,ISIC 2,ISIC 3,ISIC 4,ISIC 5
1,1,242,"('ecoinvent 3.4 cutoff', '048ac7e1860972d562e9...",7691,831198c8-1ddd-49ff-be80-9dce40d83a7e,831198c8-1ddd-49ff-be80-9dce40d83a7e_4cf8fef9-...,molecular sieve separation of naphtha,RER,1998-01-01,2017-12-31,...,Manufacturing,Manufacture of coke and refined petroleum prod...,Manufacture of refined petroleum products,Manufacture of refined petroleum products,NaN,C:Manufacturing,19:Manufacture of coke and refined petroleum p...,192:Manufacture of refined petroleum products,1920:Manufacture of refined petroleum products,NaN
2,2,251,"('ecoinvent 3.4 cutoff', '04b39baa4b5d28eb8c39...",13213,e31199c9-a24f-4cd8-b770-d6d21802ae88,e31199c9-a24f-4cd8-b770-d6d21802ae88_b658a402-...,propylene production,RER,1999-01-01,2017-12-31,...,Manufacturing,Manufacture of coke and refined petroleum prod...,Manufacture of refined petroleum products,Manufacture of refined petroleum products,NaN,C:Manufacturing,19:Manufacture of coke and refined petroleum p...,192:Manufacture of refined petroleum products,1920:Manufacture of refined petroleum products,NaN


In [11]:
AO.columns

Index(['Unnamed: 0', 'Unnamed: 0_x', 'key', '#', 'id', 'filename',
       'activityName', 'geography', 'startDate', 'endDate',
       'specialActivityType', 'technologyLevel', 'inheritance status',
       'parentActivityId', 'tags', 'synonyms', 'ISIC v4', 'ISIC code',
       'ISIC name', 'specialty production', 'constrained market',
       'accessRestrictedTo', 'dataEntryBy', 'dataGenerator', 'isActiveAuthor',
       'group', 'name', 'CPC', 'amount', 'unitName', 'productionVolumeAmount',
       'absolute PV', 'productionVolumeComment', 'price',
       'By-product classification', 'mft', 'functional unit',
       '('IPCC 2013', 'climate change', 'GWP 100a')', 'Unnamed: 0_y',
       '#activities', 'Code (ecoinvent)', 'Description (ecoinvent)',
       'ISIC level', 'Code 1', 'Code 2', 'Code 3', 'Code 4', 'Code 5',
       'Description 1', 'Description 2', 'Description 3', 'Description 4',
       'Description 5', 'ISIC 1', 'ISIC 2', 'ISIC 3', 'ISIC 4', 'ISIC 5'],
      dtype='object')

In [12]:
AOnRoW = AO[AO['geography'] != 'RoW']
AOnRoW.shape

(7986, 58)

In [13]:
# number of unique products
print(len(AO.name.unique()))
print(len(AOnRoW.name.unique()))

2851
2851


In [14]:
# number of unique product-activity combinations (what we describe in the paper as "products")
print(len((AO.name+"//"+AO.activityName).unique()))
print(len((AOnRoW.name+AOnRoW.activityName).unique()))

4208
4150


In [15]:
#These 58 activities have ONLY a RoW representation (didn't know that this exists in ecoinvent...)
set((AO.name+"//"+AO.activityName).unique()) - set((AOnRoW.name+"//"+AOnRoW.activityName).unique())

{'electricity, for reuse in municipal waste incineration only//treatment of liquid crystal display, municipal waste incineration',
 'electricity, for reuse in municipal waste incineration only//treatment of raw sewage sludge, municipal incineration',
 'electricity, for reuse in municipal waste incineration only//treatment of residue from mechanical treatment, IT accessory, municipal waste incineration',
 'electricity, for reuse in municipal waste incineration only//treatment of residue from mechanical treatment, cathode ray tube display, municipal waste incineration',
 'electricity, for reuse in municipal waste incineration only//treatment of residue from mechanical treatment, desktop computer, municipal waste incineration',
 'electricity, for reuse in municipal waste incineration only//treatment of residue from mechanical treatment, industrial device, municipal waste incineration',
 'electricity, for reuse in municipal waste incineration only//treatment of residue from mechanical trea

In [16]:
# number of unique product-activity-geography combinations
print(len((AO.name+AO.activityName+AO.geography).unique()))
print(len((AOnRoW.name+AOnRoW.activityName+AOnRoW.geography).unique()))

10803
7984


In [17]:
EI_ISIC = AO['key'].groupby(AO['Description 1']).count()
EI_ISIC.name = 'ecoinvent, w/o markets'
#EI_ISIC = EI_ISIC.astype(int)
EI_ISIC

Description 1
Administrative and support service activities                             85
Agriculture, forestry and fishing                                        808
Construction                                                             525
Electricity, gas, steam and air conditioning supply                     3162
Information and communication                                             13
Manufacturing                                                           3907
Mining and quarrying                                                     284
Other service activities                                                   2
Professional, scientific and technical activities                          4
Real estate activities                                                   200
Transportation and storage                                               248
Water supply; sewerage, waste management and remediation activities     1504
Wholesale and retail trade; repair of motor vehicles and motor

In [18]:
print('Activities in ecoinvent that have an ISIC classification:', EI_ISIC.sum())
print('Activities that lack and ISIC classification:', AO.shape[0] - EI_ISIC.sum())
# these are "electricity ... production mixes" and "Recycled Content cut-off" activities

Activities in ecoinvent that have an ISIC classification: 10760
Activities that lack and ISIC classification: 45


In [19]:
DF_EI = DF.join(EI_ISIC, 'Description').fillna(value=0).astype({'ecoinvent, w/o markets': int})
DF_EI

,Code,Description,"ecoinvent, w/o markets"
0,A,"Agriculture, forestry and fishing",808
1,B,Mining and quarrying,284
2,C,Manufacturing,3907
3,D,"Electricity, gas, steam and air conditioning s...",3162
4,E,"Water supply; sewerage, waste management and r...",1504
5,F,Construction,525
6,G,Wholesale and retail trade; repair of motor ve...,18
7,H,Transportation and storage,248
8,I,Accommodation and food service activities,0
9,J,Information and communication,13


### Ecoinvent "reduced"

1. only "ordinary transforming activities", i.e. no markets, production mixes, market groups
2. only unique product-activity combinations, i.e. ignoring p-a combinations that occur in several geographies

#### Group by product and activity to "drop" the geographies

In [20]:
AOng = AO.groupby(['name', 'activityName', 'Description 1']).count()
#AOng = AOt.groupby(['name', 'activityName']).count()
#AOng = AOt.groupby(['activityName', 'name',]).count()
AOng.reset_index(inplace=True)  
print(AOng.shape)
AOng

(4164, 58)


,name,activityName,Description 1,Unnamed: 0,Unnamed: 0_x,key,#,id,filename,geography,...,Code 5,Description 2,Description 3,Description 4,Description 5,ISIC 1,ISIC 2,ISIC 3,ISIC 4,ISIC 5
0,"1,1-difluoroethane, HFC-152a","1,1-difluoroethane production, HFC-152a",Manufacturing,2,2,2,2,2,2,2,...,0,2,2,2,0,2,2,2,2,0
1,"1,1-dimethylcyclopentane",molecular sieve separation of naphtha,Manufacturing,2,2,2,2,2,2,2,...,0,2,2,2,0,2,2,2,2,0
2,1-butanol,hydroformylation of propylene,Manufacturing,2,2,2,2,2,2,2,...,0,2,2,2,0,2,2,2,2,0
3,1-pentanol,hydroformylation of butene,Manufacturing,2,2,2,2,2,2,2,...,0,2,2,2,0,2,2,2,2,0
4,1-propanol,1-propanol production,Manufacturing,2,2,2,2,2,2,2,...,0,2,2,2,0,2,2,2,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4159,zinc oxide,zinc oxide production,Manufacturing,2,2,2,2,2,2,2,...,0,2,2,2,0,2,2,2,2,0
4160,zinc sulfide,zinc sulfide production,Manufacturing,2,2,2,2,2,2,2,...,0,2,2,2,0,2,2,2,2,0
4161,"zircon, 50% zirconium",heavy mineral sand quarry operation,Mining and quarrying,2,2,2,2,2,2,2,...,0,2,2,2,0,2,2,2,2,0
4162,zirconium oxide,zirconium oxide production,Manufacturing,2,2,2,2,2,2,2,...,0,2,2,2,0,2,2,2,2,0


In [21]:
EIng = AOng['Description 1'].groupby(AOng['Description 1']).count()
EIng.name = 'ecoinvent, w/o markets and geographies'
print(EIng.sum())
EIng

4164


Description 1
Administrative and support service activities                             26
Agriculture, forestry and fishing                                        398
Construction                                                             279
Electricity, gas, steam and air conditioning supply                      343
Information and communication                                              6
Manufacturing                                                           2059
Mining and quarrying                                                     135
Other service activities                                                   1
Professional, scientific and technical activities                          2
Real estate activities                                                    13
Transportation and storage                                               151
Water supply; sewerage, waste management and remediation activities      741
Wholesale and retail trade; repair of motor vehicles and motor

#### Join with results DF

In [22]:
DF_EIr = DF_EI.join(EIng, 'Description').fillna(value=0).astype({'ecoinvent, w/o markets and geographies': int})
DF_EIr.to_clipboard()
DF_EIr

,Code,Description,"ecoinvent, w/o markets","ecoinvent, w/o markets and geographies"
0,A,"Agriculture, forestry and fishing",808,398
1,B,Mining and quarrying,284,135
2,C,Manufacturing,3907,2059
3,D,"Electricity, gas, steam and air conditioning s...",3162,343
4,E,"Water supply; sewerage, waste management and r...",1504,741
5,F,Construction,525,279
6,G,Wholesale and retail trade; repair of motor ve...,18,10
7,H,Transportation and storage,248,151
8,I,Accommodation and food service activities,0,0
9,J,Information and communication,13,6


# EXIOBASE

### Load manually matched HIOT sectors to ISIC Code1 

In [23]:
HIOT_ISIC = pd.read_excel(f"{DATA_PATH}matching/ISIC.xlsx", sheet_name='exiobase')
HIOT_ISIC.rename(columns={"ISIC Code 1": "Code"}, inplace=True)
HIOT_ISIC.head()

,#,Code,ProductTypeName_of_hiot,product code 1,product code 2
0,1,A,Cultivation of paddy rice,c01.a,P_PARI
1,2,A,Cultivation of wheat,c01.b,P_WHEA
2,3,A,Cultivation of cereal grains nec,c01.c,P_OCER
3,4,A,"Cultivation of vegetables, fruit, nuts",c01.d,P_FVEG
4,5,A,Cultivation of oil seeds,c01.e,P_OILS


In [24]:
HIOT_nonregionalized = HIOT_ISIC.groupby('Code')['#'].nunique()
HIOT_nonregionalized

Code
A    19
B    15
C    48
D    17
E    35
F     1
G     4
H     7
I     1
J     1
K     3
L     2
M     2
N     1
O     1
P     1
Q     1
R     1
S     2
T     1
U     1
Name: #, dtype: int64

### Instead of multiplying by 48, get actual numbers of sectors (not always fully regionalized) from "hiot_results.xlsx"

In [25]:
HIOT = pd.read_excel(hiot_results, sheet_name=hiot_results_sheet)
print(HIOT.shape)
HIOT

(8200, 10)


,#,CountryCode,ProductTypeName_of_hiot,product code 1,product code 2,OriginalPhysicalUnit,GHGs-eq - tonnes,PhysicalUnit,CarbonFootprint,unit
0,1,AU,Cultivation of paddy rice,c01.a,P_PARI,tonnes,1.6,kg,1.6,kg CO2-eq/kg
1,2,AU,Cultivation of wheat,c01.b,P_WHEA,tonnes,0.307,kg,0.307,kg CO2-eq/kg
2,3,AU,Cultivation of cereal grains nec,c01.c,P_OCER,tonnes,0.266,kg,0.266,kg CO2-eq/kg
3,4,AU,"Cultivation of vegetables, fruit, nuts",c01.d,P_FVEG,tonnes,0.724,kg,0.724,kg CO2-eq/kg
4,5,AU,Cultivation of oil seeds,c01.e,P_OILS,tonnes,0.395,kg,0.395,kg CO2-eq/kg
...,...,...,...,...,...,...,...,...,...,...
8195,8196,EU-28,Activities of membership organisation n.e.c. (91),c91,P_ORGA,Meuro,294,Meuro,294,kg CO2-eq/Meuro
8196,8197,EU-28,"Recreational, cultural and sporting activities...",c92,P_RECR,Meuro,206,Meuro,206,kg CO2-eq/Meuro
8197,8198,EU-28,Other service activities (93),c93,P_OSER,Meuro,158,Meuro,158,kg CO2-eq/Meuro
8198,8199,EU-28,Private households with employed persons (95),c95,P_PRHH,Meuro,108,Meuro,108,kg CO2-eq/Meuro


#### Exclude GLO, EU-28 and rows where GWP=0 (products not produced in a country)

In [26]:
HIOTr = HIOT[HIOT['CountryCode'] != 'GLO']
HIOTr = HIOTr[HIOTr['CountryCode'] != 'EU-28']
print(HIOTr.shape)
HIOTr = HIOTr[HIOTr['CarbonFootprint'] != 0]
print(HIOTr.shape)

(7872, 10)
(6692, 10)


#### Merge with ISIC code 1

In [27]:
HIOTm = pd.merge(HIOTr, HIOT_ISIC[['ProductTypeName_of_hiot','Code']], on=["ProductTypeName_of_hiot"], how="inner")
print(HIOTm.shape)
HIOTm.head()

(6692, 11)


,#,CountryCode,ProductTypeName_of_hiot,product code 1,product code 2,OriginalPhysicalUnit,GHGs-eq - tonnes,PhysicalUnit,CarbonFootprint,unit,Code
0,1,AU,Cultivation of paddy rice,c01.a,P_PARI,tonnes,1.6,kg,1.6,kg CO2-eq/kg,A
1,493,BR,Cultivation of paddy rice,c01.a,P_PARI,tonnes,0.697,kg,0.697,kg CO2-eq/kg,A
2,657,BG,Cultivation of paddy rice,c01.a,P_PARI,tonnes,0.43,kg,0.43,kg CO2-eq/kg,A
3,985,CN,Cultivation of paddy rice,c01.a,P_PARI,tonnes,2.02,kg,2.02,kg CO2-eq/kg,A
4,1969,FR,Cultivation of paddy rice,c01.a,P_PARI,tonnes,0.31,kg,0.31,kg CO2-eq/kg,A


In [28]:
HIOT_regionalized = HIOTm.groupby('Code')['#'].nunique()
HIOT_regionalized

Code
A     761
B     437
C    2185
D     572
E    1347
F      48
G     192
H     336
I      48
J      48
K     144
L      96
M      96
N      48
O      48
P      48
Q      48
R      48
S      96
T      46
Name: #, dtype: int64

## Merge ecoinvent and EXIOBASE data

In [29]:
HIOT_nonregionalized

Code
A    19
B    15
C    48
D    17
E    35
F     1
G     4
H     7
I     1
J     1
K     3
L     2
M     2
N     1
O     1
P     1
Q     1
R     1
S     2
T     1
U     1
Name: #, dtype: int64

In [30]:
DF = pd.merge(DF_EIr, HIOT_nonregionalized, on=["Code"], how="outer").fillna(value=0)
DF.rename(columns={"#": "EXIOBASE, non-regionalized"}, inplace=True)
DF = pd.merge(DF, HIOT_regionalized, on=["Code"], how="outer").fillna(value=0)
DF.rename(columns={"#": "EXIOBASE"}, inplace=True)
DF

,Code,Description,"ecoinvent, w/o markets","ecoinvent, w/o markets and geographies","EXIOBASE, non-regionalized",EXIOBASE
0,A,"Agriculture, forestry and fishing",808,398,19,761
1,B,Mining and quarrying,284,135,15,437
2,C,Manufacturing,3907,2059,48,2.18e+03
3,D,"Electricity, gas, steam and air conditioning s...",3162,343,17,572
4,E,"Water supply; sewerage, waste management and r...",1504,741,35,1.35e+03
5,F,Construction,525,279,1,48
6,G,Wholesale and retail trade; repair of motor ve...,18,10,4,192
7,H,Transportation and storage,248,151,7,336
8,I,Accommodation and food service activities,0,0,1,48
9,J,Information and communication,13,6,1,48


# Present data

#### Ratios

In [31]:
DF['Ratio, non-regionalized'] = DF['ecoinvent, w/o markets and geographies']/DF['EXIOBASE, non-regionalized']
DF['Ratio'] = DF['ecoinvent, w/o markets']/DF['EXIOBASE']
DF

,Code,Description,"ecoinvent, w/o markets","ecoinvent, w/o markets and geographies","EXIOBASE, non-regionalized",EXIOBASE,"Ratio, non-regionalized",Ratio
0,A,"Agriculture, forestry and fishing",808,398,19,761,20.9,1.06
1,B,Mining and quarrying,284,135,15,437,9,0.65
2,C,Manufacturing,3907,2059,48,2.18e+03,42.9,1.79
3,D,"Electricity, gas, steam and air conditioning s...",3162,343,17,572,20.2,5.53
4,E,"Water supply; sewerage, waste management and r...",1504,741,35,1.35e+03,21.2,1.12
5,F,Construction,525,279,1,48,279,10.9
6,G,Wholesale and retail trade; repair of motor ve...,18,10,4,192,2.5,0.0938
7,H,Transportation and storage,248,151,7,336,21.6,0.738
8,I,Accommodation and food service activities,0,0,1,48,0,0
9,J,Information and communication,13,6,1,48,6,0.271


#### Column naming

In [32]:
column_names = [
"Code",
"Description",
"ecoinvent, w/o markets and geographies",
"EXIOBASE, non-regionalized",
"Ratio, non-regionalized",
"ecoinvent, w/o markets",
"EXIOBASE",
"Ratio",
]

In [33]:
DF = DF[column_names]
DF

,Code,Description,"ecoinvent, w/o markets and geographies","EXIOBASE, non-regionalized","Ratio, non-regionalized","ecoinvent, w/o markets",EXIOBASE,Ratio
0,A,"Agriculture, forestry and fishing",398,19,20.9,808,761,1.06
1,B,Mining and quarrying,135,15,9,284,437,0.65
2,C,Manufacturing,2059,48,42.9,3907,2.18e+03,1.79
3,D,"Electricity, gas, steam and air conditioning s...",343,17,20.2,3162,572,5.53
4,E,"Water supply; sewerage, waste management and r...",741,35,21.2,1504,1.35e+03,1.12
5,F,Construction,279,1,279,525,48,10.9
6,G,Wholesale and retail trade; repair of motor ve...,10,4,2.5,18,192,0.0938
7,H,Transportation and storage,151,7,21.6,248,336,0.738
8,I,Accommodation and food service activities,0,1,0,0,48,0
9,J,Information and communication,6,1,6,13,48,0.271


#### Totals

In [34]:
DF_FINAL = DF.append(DF.sum(numeric_only=True), ignore_index=True)
DF_FINAL.to_clipboard()
DF_FINAL

,Code,Description,"ecoinvent, w/o markets and geographies","EXIOBASE, non-regionalized","Ratio, non-regionalized","ecoinvent, w/o markets",EXIOBASE,Ratio
0,A,"Agriculture, forestry and fishing",398,19,20.9,808,761,1.06
1,B,Mining and quarrying,135,15,9,284,437,0.65
2,C,Manufacturing,2.06e+03,48,42.9,3.91e+03,2.18e+03,1.79
3,D,"Electricity, gas, steam and air conditioning s...",343,17,20.2,3.16e+03,572,5.53
4,E,"Water supply; sewerage, waste management and r...",741,35,21.2,1.5e+03,1.35e+03,1.12
5,F,Construction,279,1,279,525,48,10.9
6,G,Wholesale and retail trade; repair of motor ve...,10,4,2.5,18,192,0.0938
7,H,Transportation and storage,151,7,21.6,248,336,0.738
8,I,Accommodation and food service activities,0,1,0,0,48,0
9,J,Information and communication,6,1,6,13,48,0.271
